In [ ]:
%load_ext autoreload

In [ ]:
%autoreload

In [1]:
import os
import json
from pipeline import complete_pipeline
from sklearn.pipeline import FeatureUnion, make_pipeline, make_union

from feature_extractors import NDVIFeatureExtractor, ChannelsFeatureExtractor
from feature_extractors import LBPFeatureExtractor, GLCMFeatureExtractor
from feature_extractors import GCHFeatureExtractor
from feature_extractors import LocalFeatureExtractor, BoVW

In [2]:
import sys
sys.path.append("../")
from config import RESULTS_PATH

# Initialise feature extractors

In [3]:
feature_extractors = [
    ('freak_25_bovw_25', make_pipeline(
        LocalFeatureExtractor(descriptor='freak', threshold=25),
        BoVW(n_clusters=25)
    ))
#     ,
#     ('freak_25_bovw_10', make_pipeline(
#         LocalFeatureExtractor(descriptor='freak', threshold=25),
#         BoVW(n_clusters=10)
#     ))
#     ,
#     ('freak_25_bovw_250', make_pipeline(
#         LocalFeatureExtractor(descriptor='freak', threshold=25),
#         BoVW(n_clusters=250)
#     )),
#     ('freak_25_bovw_500', make_pipeline(
#         LocalFeatureExtractor(descriptor='freak', threshold=25),
#         BoVW(n_clusters=500)
#     )),
#     ('freak_25_bovw_500', make_pipeline(
#         LocalFeatureExtractor(descriptor='freak', threshold=25),
#         BoVW(n_clusters=1000)
#     ))
]

# evaluate performance

In [ ]:
performances = {}
for feature_name, feature_extractor in feature_extractors:
    performance = complete_pipeline(feature_extractor)
    performances[feature_name] = performance

In [ ]:
performances

In [ ]:
for feature_name, performance in performances.items():
    file_path = os.path.join(RESULTS_PATH, f'{feature_name}.json')
    with open(file_path, 'w') as f:
        json.dump(performance, f, indent=4)